# Mappings Transformation

Convert the Emacs Lisp definition of each mapping into a C# Dictionary

In [ ]:
import re

In [ ]:
# load the raw data
with open("../assets/mappings.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

len(lines)

In [ ]:
# strip whitespace, filter blank lines
lines = [l_s for l in lines if (l_s := l.strip())]
len(lines)

In [ ]:
# filter out comments
lines = [l for l in lines if not l.startswith(";;")]
len(lines)

In [ ]:
# sanity check
lines[0:5]

In [ ]:
# Extract key-value pairs
matcher = re.compile(r'"(.*?)"(?:.*?)"(.*?)"')
matcher.findall(lines[0])

In [ ]:
lookup = {k: v for line in lines for k, v in matcher.findall(line)}
lookup

In [ ]:
lookup["\\\\"] = lookup.pop("\\")

In [ ]:
# test querying capability
query_matcher = re.compile(r"^(.*?)(\d*)$")

def query(i):
    key, index = query_matcher.findall(i)[0]
    if index:
        index = int(index)
    else:
        index = 1

    if key in lookup:
        return lookup[key][index - 1]
    else:
        raise ValueError

In [ ]:
query("~=")

In [ ]:
# export
with open("../src/nthn.Agda/Lookup.cs", "w", encoding="utf-8") as f:
    f.write("""using System;
using System.Collections.Generic;
using System.Text;

namespace nthn.Agda
{
    public class AgdaLookup
    {
        public Dictionary<string, string> keyValuePairs = new Dictionary<string, string>()
        {
""")

    total = len(lookup.keys())
    for i, (k, v) in enumerate(lookup.items()):
        f.write(f"            {{\"{k}\", \"{v}\"}}")
        if i == total - 1:
            f.write("\n")
        else:
            f.write(",\n")
    f.write("        };\n    };\n}")